In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jsonls/news_prepared_train.jsonl
/kaggle/input/subset/train.csv
/kaggle/input/subset/test.csv
/kaggle/input/fake-and-real-news-dataset/True.csv
/kaggle/input/fake-and-real-news-dataset/Fake.csv


In [8]:
!python --version

Python 3.7.12


In [9]:
!pip install openai==0.25.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 5.3 MB/s eta 0:00:00
  Created wheel for openai: filename=openai-0.25.0-py3-none-any.whl size=55857 sha256=52858a70de8d7c6eab893fb74bd13538600a9ba9a9d248479fc556f600e2ec9a
  Stored in directory: /root/.cache/pip/wheels/7c/19/66/5afc0b577575f408eccb6f1b84d016b6062394c6695a0eae0d
Successfully built openai


In [10]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
from wordcloud import WordCloud
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Conv1D, MaxPool1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import openai

In [11]:
openai.api_key = "**************"

In [12]:
os.environ['OPENAI_API_KEY'] = "**********************"

In [15]:
fake = pd.read_csv("/kaggle/input/fake-and-real-news-dataset/Fake.csv")
fake.head()

title  \
0   Donald Trump Sends Out Embarrassing New Year’...   
1   Drunk Bragging Trump Staffer Started Russian ...   
2   Sheriff David Clarke Becomes An Internet Joke...   
3   Trump Is So Obsessed He Even Has Obama’s Name...   
4   Pope Francis Just Called Out Donald Trump Dur...   

                                                text subject  \
0  Donald Trump just couldn t wish all Americans ...    News   
1  House Intelligence Committee Chairman Devin Nu...    News   
2  On Friday, it was revealed that former Milwauk...    News   
3  On Christmas day, Donald Trump announced that ...    News   
4  Pope Francis used his annual Christmas Day mes...    News   

                date  
0  December 31, 2017  
1  December 31, 2017  
2  December 30, 2017  
3  December 29, 2017  
4  December 25, 2017

In [16]:
real = pd.read_csv("/kaggle/input/fake-and-real-news-dataset/True.csv")
real.head()

title  \
0  As U.S. budget fight looms, Republicans flip t...   
1  U.S. military to accept transgender recruits o...   
2  Senior U.S. Republican senator: 'Let Mr. Muell...   
3  FBI Russia probe helped by Australian diplomat...   
4  Trump wants Postal Service to charge 'much mor...   

                                                text       subject  \
0  WASHINGTON (Reuters) - The head of a conservat...  politicsNews   
1  WASHINGTON (Reuters) - Transgender people will...  politicsNews   
2  WASHINGTON (Reuters) - The special counsel inv...  politicsNews   
3  WASHINGTON (Reuters) - Trump campaign adviser ...  politicsNews   
4  SEATTLE/WASHINGTON (Reuters) - President Donal...  politicsNews   

                 date  
0  December 31, 2017   
1  December 29, 2017   
2  December 31, 2017   
3  December 30, 2017   
4  December 29, 2017

In [17]:
# Adding class Information
real["class"] = 1
fake["class"] = 0
#Combining Title and Text
real["text"] = real["title"] + " " + real["text"]
fake["text"] = fake["title"] + " " + fake["text"]
# Subject is diffrent for real and fake thus dropping it
# Aldo dropping Date, title and Publication Info of real
real = real.drop(["subject", "date","title"], axis=1)
fake = fake.drop(["subject", "date", "title"], axis=1)
#Combining both into new dataframe
data = real.append(fake, ignore_index=True)
data = data.sample(frac=1, random_state=42)[:500]
del real, fake
data

text  class
22216   BREAKING: GOP Chairman Grassley Has Had Enoug...      0
27917   Failed GOP Candidates Remembered In Hilarious...      0
25007   Mike Pence’s New DC Neighbors Are HILARIOUSLY...      0
1377   California AG pledges to defend birth control ...      1
32476  AZ RANCHERS Living On US-Mexico Border Destroy...      0
...                                                  ...    ...
33259  AWESOME! STREET ARTIST SABO TARGETS Hollywood ...      0
40054  MICHELLE OBAMA COMES OUT OF HIDING To Accuse P...      0
44036  President Obama Arrives in Cuba  USA TODAYHAVA...      0
20242  Guatemala federal auditor to probe president's...      1
22653   Trump Stops Infrastructure Speech To Go Off T...      0

[500 rows x 2 columns]

In [18]:
data = data.rename(columns={'text': 'prompt', 'class': 'completion'})

In [19]:

# truncate prompts longer than 2048 tokens
data['prompt'] = data['prompt'].str[:2048]

# add a common separator at the end of each prompt
data['prompt'] = data['prompt'].astype(str) + ' [SEP] '

# add a whitespace character to the start of the completion
data['completion'] = ' ' + data['completion'].astype(str)

data.drop_duplicates(inplace=True)


In [21]:
from sklearn.model_selection import train_test_split
# split the data into training and test sets
train_df, test_df = train_test_split(data, test_size=0.3, random_state=42)

# display the size of the training and test sets
print(f'Training set size: {len(train_df)}')
print(f'Test set size: {len(test_df)}')

Training set size: 348
Test set size: 150


In [22]:
train_df = pd.read_csv('/kaggle/input/subset/train.csv')
test_df = pd.read_csv('/kaggle/input/subset/test.csv')

In [23]:
def preprocess_data(df):
    df["text"] = " Subject: "+df["subject"]+"; Date: "+df["date"]+"; Title: `" + df["title"] + "`; Content: " + df["text"]
    df = df.drop(["subject", "date","title"], axis=1)
    df = df.rename(columns={'text': 'prompt', 'is_fake': 'completion'})
    # truncate prompts longer than 2048 tokens
    df['prompt'] = df['prompt'].str[:2048]

    # add a common separator at the end of each prompt
    df['prompt'] = df['prompt'].astype(str) + ' [SEP] '

    # add a whitespace character to the start of the completion
    df['completion'] = ' ' + df['completion'].astype(str)

    df.drop_duplicates(inplace=True)
    return df

In [24]:
train_df = preprocess_data(train_df)

In [25]:
test_df = preprocess_data(test_df)

In [26]:
train_df

prompt completion
0     Subject: politicsNews; Date: February 1, 2017...          0
1     Subject: left-news; Date: Aug 20, 2016; Title...          1
2     Subject: News; Date: September 5, 2017; Title...          1
3     Subject: worldnews; Date: September 8, 2017 ;...          0
4     Subject: worldnews; Date: November 17, 2017 ;...          0
..                                                 ...        ...
345   Subject: politicsNews; Date: April 30, 2016 ;...          0
346   Subject: left-news; Date: Jun 22, 2017; Title...          1
347   Subject: politics; Date: Apr 5, 2017; Title: ...          1
348   Subject: politicsNews; Date: March 7, 2016 ; ...          0
349   Subject: News; Date: October 13, 2016; Title:...          1

[350 rows x 2 columns]

# ChatGPT with prompt engineering

In [48]:
model_engine = "text-davinci-003"
def _get_ans_from_response(response:openai.openai_object.OpenAIObject) -> str:
    first = dict(response)['choices']
    sec = dict(first[0])
    return sec['text']

def _getter(model_engine:str = model_engine,prompt:str = "") -> str:
    # Send the request to the Chat GPT API
    response = openai.Completion.create(
                          engine=model_engine,
                          prompt=prompt,
                          max_tokens=100
                          )
    return _get_ans_from_response(response)

In [51]:
prediction = []
y_true = []
import time
from tqdm import tqdm
for i in tqdm(range(len(test_df.iloc[:]))):
    text = test_df.prompt.iloc[i]
    try:
        # See whether the server is giving proper response
        response = _getter(prompt=f"Predict whether the following piece of news is real or fake. If it is fake news return 1 else return 0. Write nothing else. : {text}")
    except Exception as e:
        #if the server is busy wait for 5s and retry again
        print('Server exception: '+str(e))
        time.sleep(5)
        continue
    if i<3:
        print(text,":",response)
    try:
        prediction.append(int(response[-1]))
        y_true.append(test_df.completion.iloc[i])
    except:
        print(response)
    
    #ChatGPT free version let's you 60+- Request Per Minute
    time.sleep(1)

  0%|          | 0/150 [00:00<?, ?it/s]

 Subject: politics; Date: Oct 13, 2017; Title: `REPORTER TO CLINTON: ‘So you still blame others more than yourself for loss?’ [VIDEO]`; Content: British TV journalist Matt Frei asked Hillary Clinton if she s still blaming other more than herself in an interview Friday: Your dynastic appeal or perhaps it was the opposite: The fact that you were called Clinton the fact that you were first lady basically trumped any novelty if you forgive the term of being the first female president of the United States,  he said.  People looked at your name and your legacy more than they looked at your gender. Clinton answered in her usual delusional way by laying out the amount of support she THINKS she received and then blaming former FBI director James Comey and the Russians for her defeat: That doesn t explain why I led all the way through, why I won the primary by 4 million votes, why I was winning, we had a great convention,  she said.  I was thought to have won all three debates. That doesn t expl

  1%|          | 1/150 [00:01<04:31,  1.83s/it]

 Subject: politicsNews; Date: June 6, 2017 ; Title: `U.S. Patent and Trademark Office head Michelle Lee resigns`; Content: (Reuters) - U.S. Patent and Trademark Office Director Michelle Lee, who has won praise from technology companies for taking steps to minimize abusive patent litigation, resigned from her position on Tuesday, a spokesman for the agency has confirmed. She joined the agency in 2012 and became interim director in 2013 before being formerly nominated as director by then-President Barack Obama in 2014.  Some inventors and patent licensing companies had opposed Lee, who was previously an in-house lawyer for Alphabet Inc’s Google. The patent office is a division of the U.S. Department of Commerce that reviews applications for patents and trademarks. The agency is also empowered to cancel patents it has issued. Lee has been viewed favorably by the technology industry for ushering in more stringent standards for software patents. More than 50 companies, including Facebook In

  1%|▏         | 2/150 [00:03<04:04,  1.65s/it]

 Subject: politics; Date: Jul 15, 2016; Title: `POLICE IN GERMANY BEGIN RAIDS On Homes Of Facebook Users Who Post “Hate Speech” Against Refugees`; Content: If Facebook has aligned themselves with Germany to restrict the free speech of its users, what makes us so sure they won t do the same thing in the US with Obama or (God forbid) a Hillary presidency? And wouldn t Germany be better off focusing their policing efforts on the over 1 million freeloaders and terrorists they ve just admitted into Germanista?Police in Germany have started carrying out raids in peoples homes in an attempt to prosecute people for inciting  hate speech  on Facebook.A press release from Germany s federal police agency confirmed that around 60 homes were searched this week as a result of people posting messages deemed as extreme by the German authorities. The action carried out today shows that the authorities are acting firmly against hate on the internet, which has grown considerably in the wake of the refuge

 67%|██████▋   | 100/150 [03:19<01:26,  1.74s/it]



Fake News


100%|██████████| 150/150 [04:50<00:00,  1.93s/it]


In [52]:
from sklearn.metrics import classification_report
y_true = np.char.strip(np.array(y_true).astype(str))
prediction = np.char.strip(np.array(prediction).astype(str))
print(classification_report(y_true,prediction))

              precision    recall  f1-score   support

           0       0.48      1.00      0.65        60
           1       1.00      0.28      0.44        89

    accuracy                           0.57       149
   macro avg       0.74      0.64      0.55       149
weighted avg       0.79      0.57      0.52       149



# Fine tuned GPT 3

In [44]:
train_df.to_json("news.jsonl", orient='records', lines=True)

In [45]:
!openai tools fine_tunes.prepare_data -f news.jsonl -q

Analyzing...

- Your file contains 350 prompt-completion pairs
- Based on your data it seems like you're trying to fine-tune a model for classification
- For classification, we recommend you try one of the faster and cheaper models, such as `ada`
- For classification, you can estimate the expected model performance by keeping a held out dataset, which is not used for training
- All prompts end with suffix ` [SEP] `
- All prompts start with prefix ` Subject: `

No remediations found.
- [Recommended] Would you like to split into training and validation set? [Y/n]: Y


Your data will be written to a new JSONL file. Proceed [Y/n]: Y

Wrote modified files to `news_prepared_train.jsonl` and `news_prepared_valid.jsonl`
Feel free to take a look!

Now use that file when fine-tuning:
> openai api fine_tunes.create -t "news_prepared_train.jsonl" -v "news_prepared_valid.jsonl" --compute_classification_metrics --classification_positive_class " 1"

After you’ve fine-tuned a model, remember that your

In [64]:
!openai --api-key '******************' api fine_tunes.create -t "file-JI8MkWsKzSxy2pvirFlkyrro" -v "file-NDefDVRvj4d6Bi8MSAGwDRf6" --compute_classification_metrics --classification_positive_class " 1" -m ada

[2023-05-02 15:54:31,156] message='Request to OpenAI API' method=get path=https://api.openai.com/v1/files/file-JI8MkWsKzSxy2pvirFlkyrro
[2023-05-02 15:54:31,284] message='OpenAI API response' path=https://api.openai.com/v1/files/file-JI8MkWsKzSxy2pvirFlkyrro processing_ms=15 request_id=f6ffe49ae1d62322bb816916862b36e0 response_code=200
[2023-05-02 15:54:31,285] message='Request to OpenAI API' method=get path=https://api.openai.com/v1/files/file-NDefDVRvj4d6Bi8MSAGwDRf6
[2023-05-02 15:54:31,372] message='OpenAI API response' path=https://api.openai.com/v1/files/file-NDefDVRvj4d6Bi8MSAGwDRf6 processing_ms=12 request_id=17591da050c79adbf331496b8580ec25 response_code=200
[2023-05-02 15:54:31,373] message='Request to OpenAI API' method=post path=https://api.openai.com/v1/fine-tunes
[2023-05-02 15:54:31,563] message='OpenAI API response' path=https://api.openai.com/v1/fine-tunes processing_ms=115 request_id=4685afb3417345a802129bb7bfac6ec1 response_code=200
Created fine-tune: ft-k0arzZg4gRFc

In [66]:
!openai api fine_tunes.follow -i ft-k0arzZg4gRFcLPYT585RpgXO

[2023-05-02 16:47:52,026] message='Request to OpenAI API' method=get path=https://api.openai.com/v1/fine-tunes/ft-k0arzZg4gRFcLPYT585RpgXO/events?stream=true
[2023-05-02 16:47:52,164] message='OpenAI API response' path=https://api.openai.com/v1/fine-tunes/ft-k0arzZg4gRFcLPYT585RpgXO/events?stream=true processing_ms=22 request_id=47f66e347b75b35f557df5a2d5e50e61 response_code=200
[2023-05-02 15:54:31] Created fine-tune: ft-k0arzZg4gRFcLPYT585RpgXO
[2023-05-02 16:07:31] Fine-tune costs $0.16
[2023-05-02 16:07:31] Fine-tune enqueued. Queue number: 2
[2023-05-02 16:13:44] Fine-tune is in the queue. Queue number: 1
[2023-05-02 16:14:28] Fine-tune is in the queue. Queue number: 0
[2023-05-02 16:15:43] Fine-tune started
[2023-05-02 16:16:44] Completed epoch 1/4
[2023-05-02 16:17:32] Completed epoch 2/4
[2023-05-02 16:18:18] Completed epoch 3/4
[2023-05-02 16:19:04] Completed epoch 4/4
[2023-05-02 16:19:27] Uploaded model: ada:ft-personal-2023-05-02-16-19-26
[2023-05-02 16:19:28] Uploaded resu

In [57]:
ft_model = 'ada:ft-personal-2023-05-02-16-19-26'
prediction = []
y_true = []
import time
from tqdm import tqdm
for i in tqdm(range(len(test_df.iloc[:]))):
    text = test_df.prompt.iloc[i]
    try:
        # See whether the server is giving proper response
        response = openai.Completion.create(model=ft_model, prompt=text, max_tokens=1, temperature=0, logprobs=2)
        response = response['choices'][0]['text']
    except Exception as e:
        #if the server is busy wait for 5s and retry again
        print('Server exception: '+str(e))
        time.sleep(5)
        continue
    if i<3:
        print(text,":",response)
    try:
        prediction.append(int(response[-1]))
        y_true.append(test_df.completion.iloc[i])
    except:
        print(response)

    time.sleep(1)

  0%|          | 0/150 [00:00<?, ?it/s]

 Subject: politics; Date: Oct 13, 2017; Title: `REPORTER TO CLINTON: ‘So you still blame others more than yourself for loss?’ [VIDEO]`; Content: British TV journalist Matt Frei asked Hillary Clinton if she s still blaming other more than herself in an interview Friday: Your dynastic appeal or perhaps it was the opposite: The fact that you were called Clinton the fact that you were first lady basically trumped any novelty if you forgive the term of being the first female president of the United States,  he said.  People looked at your name and your legacy more than they looked at your gender. Clinton answered in her usual delusional way by laying out the amount of support she THINKS she received and then blaming former FBI director James Comey and the Russians for her defeat: That doesn t explain why I led all the way through, why I won the primary by 4 million votes, why I was winning, we had a great convention,  she said.  I was thought to have won all three debates. That doesn t expl

  1%|          | 1/150 [00:01<02:57,  1.19s/it]

 Subject: politicsNews; Date: June 6, 2017 ; Title: `U.S. Patent and Trademark Office head Michelle Lee resigns`; Content: (Reuters) - U.S. Patent and Trademark Office Director Michelle Lee, who has won praise from technology companies for taking steps to minimize abusive patent litigation, resigned from her position on Tuesday, a spokesman for the agency has confirmed. She joined the agency in 2012 and became interim director in 2013 before being formerly nominated as director by then-President Barack Obama in 2014.  Some inventors and patent licensing companies had opposed Lee, who was previously an in-house lawyer for Alphabet Inc’s Google. The patent office is a division of the U.S. Department of Commerce that reviews applications for patents and trademarks. The agency is also empowered to cancel patents it has issued. Lee has been viewed favorably by the technology industry for ushering in more stringent standards for software patents. More than 50 companies, including Facebook In

  1%|▏         | 2/150 [00:02<02:52,  1.17s/it]

 Subject: politics; Date: Jul 15, 2016; Title: `POLICE IN GERMANY BEGIN RAIDS On Homes Of Facebook Users Who Post “Hate Speech” Against Refugees`; Content: If Facebook has aligned themselves with Germany to restrict the free speech of its users, what makes us so sure they won t do the same thing in the US with Obama or (God forbid) a Hillary presidency? And wouldn t Germany be better off focusing their policing efforts on the over 1 million freeloaders and terrorists they ve just admitted into Germanista?Police in Germany have started carrying out raids in peoples homes in an attempt to prosecute people for inciting  hate speech  on Facebook.A press release from Germany s federal police agency confirmed that around 60 homes were searched this week as a result of people posting messages deemed as extreme by the German authorities. The action carried out today shows that the authorities are acting firmly against hate on the internet, which has grown considerably in the wake of the refuge

100%|██████████| 150/150 [04:00<00:00,  1.61s/it]


In [58]:
y_true = np.char.strip(np.array(y_true).astype(str))
prediction = np.char.strip(np.array(prediction).astype(str))
print(classification_report(y_true,prediction))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        60
           1       1.00      1.00      1.00        90

    accuracy                           1.00       150
   macro avg       1.00      1.00      1.00       150
weighted avg       1.00      1.00      1.00       150

